In [1]:
##########################################################
##  -> Generation of spiking traces for the 10,000      ##
##     MNIST test samples and writing storage files     ##
##     COE -> AMD VHDL                                  ##
##                                                      ##
## !! one file CEO output                            !! ##
##                                                      ##
##########################################################
## USE this programm anly for the VHDL code             ##
##              With HUB75 - 15400 COE                  ##
##########################################################
##          MNISTWriteFileSptXOR(HUB75-70-µmac)         ##
##          -----------------------------------         ##
##                                                      ##
## -> NPY inputs - Spiking traces (20+200)              ##
##                                                      ##
## -> each NPY -> the MNIST sample index                ##
##               -> the full 28px28p representation     ##
##               -> the 220 spiking time representations##
##                                                      ##
## -> the program takes as input a properly formatted   ##
## file (MNISTWriteFile2NPY_GIF-01)                     ##
## of type npy (it contains 220 matrices of 28 by       ##
## 28 bits) and generates two files:                    ##
##   -> an npy of 220 matrices of 28 by 28 -> XOR       ##
##   -> a COE of 220 matrices of 28 by 28 -> XOR        ##
## using the XOR function between two matrices          ##
## helps avoid two consecutive spikes at '1'            ##
##                                                      ##
## all source NPY files in ../*/npy                     ##
## all output COE files in ../*/coexor                  ##
##                                                      ##
## * => base root e.g. /MNIST -> to be specified        ## 
##                                                      ##
##   07-08-2025              Pascal Harmeling (2025)    ##
##########################################################
# for reading and processing the NPY file
using NPZ

using MLDatasets
train_x, train_y = MNIST(split=:test)[:]

(features = Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], targets = [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [2]:
#######################################################################
# to be defined for the start index and the number of images created
# for the COE file
#######################################################################
#                              Warning
#
# - size of the area => MaxImg
#      the BRAM may be too small to hold the file -> reduce MaxImg
# - StartImg+MaxImg<=10000 
# - StSize => always '1', '2', '4', '5' or 10
#             - '1'  -> 20 black + 200 spiking traces - '220'
#             - '2'  -> 10 black + 100 spiking traces - '110'
#             - '4'  ->  5 black +  50 spiking traces - '055'
#             - '5'  ->  4 black +  40 spiking traces - '044'
#             - '10' ->  2 black +  20 spiking traces - '022'
#
#######################################################################

const StartImg = 1 
const MaxImg = 70
const StSize = 1

#################################################
## analysis of the constant values             ##
#################################################    
if ((StartImg+MaxImg-1)> (last(size(train_x))))
 error("ERROR -> The value of (StartImg+MaxImg) is greater than the maximum size of train_x, program stopped.")
end
 
if ((StSize != 1) && (StSize != 2) && (StSize != 4) && (StSize != 5) && (StSize != 10))
error("ERROR -> The value of StSize must be '1','2','4' or '5', program stopped.")
end

NbrBlack::Int32 = 20 / StSize
NbrSpiking::Int32 = 200 / StSize

###########################################################
## Set vairiables to read,write npy and coexor files     ##
###########################################################
# Read a .npy file
a = "../MNIST/" ## -> adapt as needed

##constante 
const b = "NPY/"
const c = "NPYXOR/" 
const d = "COEXOR/"

# Définir les chemin de travail
dir_pathread = a * b
#dir_pathwritenpyxor = a * c
dir_pathwritecoexor = a * d

#################################################
## analyse des chemins et noms de fichiers     ##
#################################################    
if !(isdir(dir_pathread))
    error("ERROR -> The directory for reading files does not exist -> end of program.")
end

if !(isdir(dir_pathwritecoexor))
    error("ERROR -> The directory for writing files does not exist -> end of program.")
end

#################################################
## creation and initialization of working      ##
## variables                                   ##
#################################################
XORTSspikes = zeros(Bool, 28,28,220)
array = zeros(Bool, 28,28,220)

#################################################
## main program -                              ##
##                creation of the *.COE file   ##
#################################################
filename = dir_pathwritecoexor *"spiking_number_" * lpad(string(StartImg), 5, '0') * "to" * lpad(string(StartImg+MaxImg-1), 5, '0') * "(Size_" * lpad(string(NbrBlack+NbrSpiking),3, '0')* ").coe"
file=open(filename, "w")
write(file, string(";----------------------------------------------------------------------\n"))
write(file, string(";---\n"))
write(file, string(";--- Package producted by julia - creating the Spiking trace matrix XOR\n"))
write(file, string(";---\n"))
write(file, string(";----------------------------------------------Pascal Harmeling 2025---\n"))
write(file, string(";\n"))
write(file, string("memory_initialization_radix=2; binary \n"))
write(file, string("memory_initialization_vector=\n"))

global Img

## Main Loop
for  Img in StartImg:(StartImg+MaxImg-1)
    Num = string(bitstring(train_y[Img]))[61:64]
    filename = dir_pathread  *"spiking_number" * lpad(string(Img), 5, '0') * ".npy"
    
    if !(isfile(filename))
        println("ERROR -> $filename")
        error("ERROR -> le fichier *.npy n'existe pas")
    end
    
    array = npzread(filename)
    XORTSspikes = zeros(Bool, 28,28,220)
    XORTSspikes[:,:,(NbrBlack+1)] = array[:,:,21]
    XORTSspikes[:,:,(NbrBlack+NbrSpiking)] = >=(0.5).(train_x[:,:,Img]) ## set last display to check answer

    #generate spiking trace without two spikes ...
    for i in 2:(NbrSpiking-1)
        XORTSspikes[:,:,(NbrBlack+i)] = (XORTSspikes[:,:,(NbrBlack+i-1)] .⊻  array[:,:,(20+i)] ) .& array[:,:,(20+i)]
    end

    ## write all  Spiking Trace matrix XOR (x,y,Img)
    for z=1:1:NbrBlack+NbrSpiking
        write(file, string(";tableau $z de l'image $Img à pour solution : $(train_y[Img]) \n"))
        ## ajoute les informations pour la micro machine
        if (z==1)
            write(file, string("10"))
        elseif (z==(NbrBlack+NbrSpiking))
            write(file, string("01"))
        else
            write(file, string("00"))
        end
        ##add Num
        write(file, Num)
            
        XORTSspikes[:,:,z]= transpose(XORTSspikes[:,:,z])
        for y=28:-1:1
            for x=28:-1:1
                if XORTSspikes[y,x,z]==false
                    write(file, string("0"))
                else
                    write(file, string("1"))
                end
            end
        end
        if (!((z==(NbrBlack+NbrSpiking)) && (Img==(StartImg+MaxImg-1))))
            write(file, string(",\n"))
        end
    end
end
close(file)
println(filename)


../MNIST/NPY/spiking_number00070.npy
